In [1]:
#Project Submission File - Team4
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,036 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,304 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,306 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubunt

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameFunctions").getOrCreate()

In [3]:
# Please ensure you have the Sleep_Efficiency_Updated.csv uploaded to the local instance on colab
from pyspark import SparkFiles
df = ("Sleep_Efficiency_Updated.csv")
spark.sparkContext.addFile(df)
df = spark.read.csv(SparkFiles.get("Sleep_Efficiency_Updated.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)

# Show DataFrame
df.show()

+---+---+------+----------------+----------------+--------------+----------------+--------------------+---------------------+----------------------+----------+--------------------+-------------------+--------------+------------------+
| ID|Age|Gender|         Bedtime|     Wakeup time|Sleep duration|Sleep efficiency|REM sleep percentage|Deep sleep percentage|Light sleep percentage|Awakenings|Caffeine consumption|Alcohol consumption|Smoking status|Exercise frequency|
+---+---+------+----------------+----------------+--------------+----------------+--------------------+---------------------+----------------------+----------+--------------------+-------------------+--------------+------------------+
|  1| 65|Female|06/03/2021 01:00|06/03/2021 07:00|             6|            0.88|                  18|                   70|                    12|         0|                   0|                  0|           Yes|                 3|
|  2| 69|  Male|05/12/2021 02:00|05/12/2021 09:00|          

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('sleep_efficiency')

In [5]:
#display all data from the view to verify with the dataframe
#the data analysis through sql includes all data outliers present in the dataset
Data_sleep_efficiency = """
SELECT
 *
FROM sleep_efficiency
"""
spark.sql(Data_sleep_efficiency).show()

+---+---+------+----------------+----------------+--------------+----------------+--------------------+---------------------+----------------------+----------+--------------------+-------------------+--------------+------------------+
| ID|Age|Gender|         Bedtime|     Wakeup time|Sleep duration|Sleep efficiency|REM sleep percentage|Deep sleep percentage|Light sleep percentage|Awakenings|Caffeine consumption|Alcohol consumption|Smoking status|Exercise frequency|
+---+---+------+----------------+----------------+--------------+----------------+--------------------+---------------------+----------------------+----------+--------------------+-------------------+--------------+------------------+
|  1| 65|Female|06/03/2021 01:00|06/03/2021 07:00|             6|            0.88|                  18|                   70|                    12|         0|                   0|                  0|           Yes|                 3|
|  2| 69|  Male|05/12/2021 02:00|05/12/2021 09:00|          

In [6]:
# Now Let's try answering some questions about our data through the power of sql
# 3. What is the average sleep efficiency of men and women?
avg_sleep_efficiency_by_gender = """
SELECT
  Gender,
  (avg(`sleep efficiency`)) AS avg_sleep_efficiency
FROM sleep_efficiency
GROUP BY Gender
ORDER BY Gender DESC
"""
spark.sql(avg_sleep_efficiency_by_gender).show()


+------+--------------------+
|Gender|avg_sleep_efficiency|
+------+--------------------+
|  Male|  0.7929467084639502|
|Female|  0.7873913043478261|
+------+--------------------+



In [19]:
# 4. What is the maximum caffine consumption of people with sleep efficiency >85%?
max_sleep_efficiency =  """
SELECT
  max(`Caffeine consumption`) AS max_Caffeine
FROM sleep_efficiency
WHERE `Caffeine consumption` is not NULL
and  `sleep efficiency` > .85
"""
spark.sql(max_sleep_efficiency).show()


+------------+
|max_Caffeine|
+------------+
|          75|
+------------+



In [20]:
# 5. What is the average caffine consumption of people with sleep efficiency >85%?
avg_sleep_efficiency =  """
SELECT
  avg(`Caffeine consumption`) AS avg_Caffeine
FROM sleep_efficiency
WHERE `Caffeine consumption` is not NULL
and  `sleep efficiency` > .85
"""
spark.sql(avg_sleep_efficiency).show()

+------------------+
|      max_Caffeine|
+------------------+
|28.630705394190873|
+------------------+



In [21]:
# 6. What is the unique combination of sleep efficiency by exercise frequency for the top 20 exercise frequencies
sleep_eff_by_exercise = """
SELECT Distinct
  `sleep efficiency` AS Sleep_efficiency,
  `Exercise frequency` AS Exercise_frequency
FROM sleep_efficiency
order by Exercise_frequency desc
"""
spark.sql(sleep_eff_by_exercise).show(20)

+----------------+------------------+
|Sleep_efficiency|Exercise_frequency|
+----------------+------------------+
|            0.52|                 5|
|            0.93|                 5|
|             0.9|                 5|
|            0.92|                 5|
|            0.85|                 5|
|            0.87|                 5|
|            0.91|                 5|
|            0.82|                 4|
|            0.91|                 4|
|             0.8|                 4|
|            0.84|                 4|
|             0.9|                 4|
|            0.52|                 4|
|            0.96|                 4|
|            0.87|                 4|
|            0.77|                 4|
|            0.67|                 4|
|            0.76|                 4|
|            0.95|                 4|
|            0.93|                 4|
+----------------+------------------+
only showing top 20 rows



In [22]:
# 7. What is the maximum sleep efficiency by smoking status and gender

sleep_efficiency_smoking_status_by_gender = """
SELECT
  max(`Sleep efficiency`),
  `smoking status` as smoking_status,
  gender
FROM sleep_efficiency
GROUP BY smoking_status,gender
"""
spark.sql(sleep_efficiency_smoking_status_by_gender).show()


+---------------------+--------------+------+
|max(Sleep efficiency)|smoking_status|gender|
+---------------------+--------------+------+
|                 0.98|            No|Female|
|                 0.99|            No|  Male|
|                 0.95|           Yes|Female|
|                 0.97|           Yes|  Male|
+---------------------+--------------+------+



In [23]:
# 8. What is the average sleep efficiency by smoking status and gender

sleep_efficiency_smoking_status_by_gender = """
SELECT
  avg(`Sleep efficiency`),
  `smoking status` as smoking_status,
  gender
FROM sleep_efficiency
GROUP BY smoking_status,gender
"""
spark.sql(sleep_efficiency_smoking_status_by_gender).show()

+---------------------+--------------+------+
|avg(Sleep efficiency)|smoking_status|gender|
+---------------------+--------------+------+
|   0.8151801801801803|            No|Female|
|   0.8175935828877003|            No|  Male|
|    0.758030303030303|           Yes|  Male|
|    0.707272727272727|           Yes|Female|
+---------------------+--------------+------+



In [24]:
# 9. Average sleep by gender and smoking status including outliers
sleep_efficiency_smoking_status_by_gender = """
SELECT
  avg(`Sleep efficiency`),
  `smoking status` as smoking_status,
  gender
FROM sleep_efficiency
GROUP BY smoking_status,gender
"""
spark.sql(sleep_efficiency_smoking_status_by_gender).show()

+---------------------+--------------+------+
|avg(Sleep efficiency)|smoking_status|gender|
+---------------------+--------------+------+
|   0.8151801801801803|            No|Female|
|   0.8175935828877003|            No|  Male|
|    0.758030303030303|           Yes|  Male|
|    0.707272727272727|           Yes|Female|
+---------------------+--------------+------+



In [25]:
# 10. What is the unique combination of sleep efficiency by exercise frequency for the top 20 exercise frequencies viewed by gender

sleep_eff_by_exercise = """
SELECT Distinct
  `sleep efficiency` AS Sleep_efficiency,
  `Exercise frequency` AS Exercise_frequency,
  gender
FROM sleep_efficiency
group by Sleep_efficiency,Exercise_frequency,gender
order by Exercise_frequency desc
"""
spark.sql(sleep_eff_by_exercise).show(20)

+----------------+------------------+------+
|Sleep_efficiency|Exercise_frequency|gender|
+----------------+------------------+------+
|            0.93|                 5|Female|
|            0.52|                 5|Female|
|            0.91|                 5|Female|
|            0.92|                 5|Female|
|            0.85|                 5|Female|
|            0.87|                 5|Female|
|             0.9|                 5|Female|
|            0.93|                 4|Female|
|            0.88|                 4|  Male|
|             0.9|                 4|Female|
|            0.91|                 4|Female|
|            0.82|                 4|Female|
|             0.8|                 4|Female|
|            0.95|                 4|  Male|
|            0.96|                 4|Female|
|            0.81|                 4|Female|
|            0.87|                 4|Female|
|            0.92|                 4|  Male|
|            0.89|                 4|  Male|
|         